In [1]:
BUCKET_NAME='test-xwh01'
OBS_BASE_PATH=BUCKET_NAME

from modelarts.session import Session
session = Session()

session.download_data(bucket_path="test-xwh01/test-machine/ratings.dat", path="./ratings.dat")
session.download_data(bucket_path="test-xwh01/test-machine/users.dat", path="./users.dat")
session.download_data(bucket_path="test-xwh01/test-machine/movies.dat", path="./movies.dat")
# import same usefull libraries
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
movie_rating = pd.read_csv('./ratings.dat')

Successfully download file test-xwh01/test-machine/ratings.dat from OBS to local ./ratings.dat
Successfully download file test-xwh01/test-machine/users.dat from OBS to local ./users.dat
Successfully download file test-xwh01/test-machine/movies.dat from OBS to local ./movies.dat


In [2]:
movie_rating_cols = ['user_id',  'movie_id', 'rating', 'unix_tiemstamp'] 
movie_rating = pd.read_csv('./ratings.dat',sep = '::', names = movie_rating_cols, parse_dates = True)
movie_rating.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,user_id,movie_id,rating,unix_tiemstamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
movie_rating.nunique()

user_id             6040
movie_id            3706
rating                 5
unix_tiemstamp    458455
dtype: int64

In [4]:
user=pd.read_csv('./users.dat',sep='::',names=['UserID','Gender','Age','Occupation','Zip-code'])
user.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
user.shape

(6040, 5)

In [6]:
movie=pd.read_csv('./movies.dat',sep='::',names=['MovieID','Title','Genres'])
movie.head()

/home/ma-user/anaconda3/envs/XGBoost-Sklearn/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movie.shape

(3883, 3)

In [8]:
movie_rating.drop("unix_tiemstamp",inplace = True, axis = 1)
movie_rating.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [9]:
# 创建用户电影矩阵
movie_rating_matrix = movie_rating.pivot_table(index = ['movie_id'], columns = ['user_id'],
                                              values = 'rating').reset_index(drop = True)
movie_rating_matrix.fillna(0, inplace = True)
cmu = movie_rating_matrix
cmu.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 利用余弦算法获得电影相似度矩阵
movie_similarity = 1 - pairwise_distances(movie_rating_matrix.values, metric = "cosine")
np.fill_diagonal(movie_similarity,0)
# 将相似度填回相似矩阵
movie_rating_matrix = pd.DataFrame(movie_similarity)
movie_rating_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,0.000000,0.390349,0.267943,0.178789,0.256569,0.347373,0.301490,0.125709,0.106620,0.377459,...,0.099502,0.020966,0.084105,0.081826,0.045949,0.309676,0.186633,0.093479,0.042829,0.182691
1,0.390349,0.000000,0.240946,0.155457,0.249970,0.244827,0.262772,0.196521,0.158469,0.386200,...,0.061819,0.015209,0.075310,0.095573,0.074271,0.213650,0.140781,0.087013,0.026063,0.122185
2,0.267943,0.240946,0.000000,0.192788,0.308290,0.187020,0.292230,0.092122,0.128378,0.245601,...,0.038492,0.065507,0.049512,0.087377,0.050985,0.190575,0.104837,0.062258,0.010073,0.097786
3,0.178789,0.155457,0.192788,0.000000,0.271990,0.125170,0.220024,0.049554,0.060334,0.133707,...,0.055486,0.053300,0.002227,0.025278,0.025204,0.118902,0.096318,0.022588,0.024769,0.095154
4,0.256569,0.249970,0.308290,0.271990,0.000000,0.148114,0.305107,0.095512,0.138392,0.237681,...,0.026632,0.083898,0.046399,0.047542,0.016156,0.174554,0.092403,0.051633,0.010750,0.112835


In [11]:
# 获得TOP-N推荐
user_inp = "Toy Story (1995)"
inp = movie[movie['Title'] == user_inp].index.tolist()
inp = inp[0]
movie['similarity'] = movie_rating_matrix.iloc[inp]
movie.columns = ['MovieID','Title','Genres','similarity']
movie.head()

,MovieID,Title,Genres,similarity
0,1,Toy Story (1995),Animation|Children's|Comedy,0.000000
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.390349
2,3,Grumpier Old Men (1995),Comedy|Romance,0.267943
3,4,Waiting to Exhale (1995),Comedy|Drama,0.178789
4,5,Father of the Bride Part II (1995),Comedy,0.256569
